O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Preparação de Dados

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

# Carregar os dados
data = pd.read_csv("datasets/car_data.csv")

# Visualizar os dados
print(data.head(), "\n")
print("\n", data.info())


        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

In [ ]:
# Codificação de variáveis categóricas usando método implícito da biblioteca LightGBM
categorical_features = data.select_dtypes(include=["object"]).columns.tolist()
for col in categorical_features:
    data[col] = data[col].astype("category")
print(data.info())

# Para os outros modelos, usaremos label-encoding
le = LabelEncoder()
data_encoded = data.copy()
for col in categorical_features:
    data_encoded[col] = le.fit_transform(data_encoded[col].astype(str))
print(data_encoded.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   DateCrawled        354369 non-null  category
 1   Price              354369 non-null  int64   
 2   VehicleType        316879 non-null  category
 3   RegistrationYear   354369 non-null  int64   
 4   Gearbox            334536 non-null  category
 5   Power              354369 non-null  int64   
 6   Model              334664 non-null  category
 7   Mileage            354369 non-null  int64   
 8   RegistrationMonth  354369 non-null  int64   
 9   FuelType           321474 non-null  category
 10  Brand              354369 non-null  category
 11  NotRepaired        283215 non-null  category
 12  DateCreated        354369 non-null  category
 13  NumberOfPictures   354369 non-null  int64   
 14  PostalCode         354369 non-null  int64   
 15  LastSeen           354369 non-null

## Treinamento do modelo

In [3]:
# Definir função de iteração para hiperparâmetros
def train_and_evaluate_model(
    model, features_train, features_valid, target_train, target_valid
):
    param_grids = {
        "linearregression": [
            {"fit_intercept": True},
        ],
        "randomforestregressor": [
            {
                "n_estimators": 50,
                "max_depth": 10,
                "n_jobs": -1,
            },
            {"n_estimators": 100, "max_depth": 15, "n_jobs": -1},
            {"n_estimators": 100, "max_depth": 8, "n_jobs": -1},
        ],
        "lgbmregressor": [
            {
                "n_estimators": 50,
                "learning_rate": 0.1,
                "max_depth": 8,
                "num_leaves": 31,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 100,
                "learning_rate": 0.1,
                "max_depth": 10,
                "num_leaves": 50,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 80,
                "learning_rate": 0.15,
                "max_depth": 6,
                "num_leaves": 64,
                "n_jobs": -1,
                "verbosity": -1,
            },
        ],
    }

    model_name = type(model).__name__.lower()
    param_grid = param_grids.get(model_name, [])

    best_mse = float("inf")
    best_r2 = float("-inf")
    best_params = None

    if len(param_grid) == 0:
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        mse = mean_squared_error(target_valid, predictions)
        r2 = r2_score(target_valid, predictions)
        print(f"Model: {type(model).__name__}, MSE: {mse:.2f}, R2: {r2:.4f}")
        return mse, r2, None

    for param_set in param_grid:
        model.set_params(**param_set)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        mse = mean_squared_error(target_valid, predictions)
        r2 = r2_score(target_valid, predictions)
        if mse < best_mse:
            best_mse = mse
            best_r2 = r2
            best_params = param_set

    print(
        f"Model: {type(model).__name__}, Best MSE: {best_mse:.2f}, Best R2: {best_r2:.4f}"
    )
    print(f"Best params: {best_params}")
    return best_mse, best_r2, best_params


In [4]:
%%time
# Dividir os dados em treinamento e teste
X = data_encoded.drop("Price", axis=1)
y = data_encoded["Price"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


CPU times: total: 188 ms
Wall time: 186 ms


In [5]:
%%time
# Regressão Linear
lr_model = LinearRegression()
lr_mse, lr_r2, lr_params = train_and_evaluate_model(
    lr_model, X_train, X_test, y_train, y_test
)


Model: LinearRegression, Best MSE: 14494183.62, Best R2: 0.2999
Best params: {'fit_intercept': True}
CPU times: total: 219 ms
Wall time: 160 ms


*Observações: MSE de 14494183.62 e R2 de 0.2999 indicam que o modelo é pouco eficiente em predizer os dados de preço com as features disponíveis. Não obstante, o Wall time de 160 ms demonstra que a regressão linear possui um treinamento extremamente rápido!*

In [6]:
%%time
# Floresta Aleatória
rf_model = RandomForestRegressor(random_state=42)
rf_mse, rf_r2, rf_params = train_and_evaluate_model(
    rf_model, X_train, X_test, y_train, y_test
)


Model: RandomForestRegressor, Best MSE: 3292863.87, Best R2: 0.8409
Best params: {'n_estimators': 100, 'max_depth': 15, 'n_jobs': -1}
CPU times: total: 12min 46s
Wall time: 1min 12s


*Observações: MSE de 3292863.87 e R2 de 0.8409 indicam que o modelo é muito mais eficiente em predizer os dados de preço com as features disponíveis, quando comparado com a Regressão Linear. Não obstante, o Wall time de 1min e 12s demonstra que a floresta aleatória possui um treinamento muito mais demorado!*

In [ ]:
%%time
# Preparar dados para LightGBM
X_lgbm = data.copy().drop("Price", axis=1)
y_lgbm = data["Price"]


X_lgbm_train, X_lgbm_test, y_lgbm_train, y_lgbm_test = train_test_split(
    X_lgbm, y_lgbm, test_size=0.2, random_state=42
)


# LightGBM com categorical encoding nativo
lgbm_model = LGBMRegressor(random_state=42, verbosity=-1)
lgbm_mse, lgbm_r2, lgbm_params = train_and_evaluate_model(
    lgbm_model, X_lgbm_train, X_lgbm_test, y_lgbm_train, y_lgbm_test
)


Model: LGBMRegressor, Best MSE: 3260348.28, Best R2: 0.8425
Best params: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 50, 'n_jobs': -1, 'verbosity': -1}
CPU times: total: 1min 20s
Wall time: 7.45 s


*Observações: MSE de 3260348.28 e R2 de 0.8425 indicam que o o gradient boosting alcança ainda melhores resultados que a floresta aleatória, consumindo uma fração do tempo de treinamento, com Wall time de 7.45s! Outra vantagem é o reconhecimento implícito de variáveis categóricas, liberando a necessidade de pré codificá-las como fizemos para a regressão linear e para a floresta aleatória* 